<a href="https://colab.research.google.com/github/SpyderWebStudios/SpyderWebBlogs/blob/main/Spotify_ETL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spotify APIs: Extract - Transform - Load

## This program fetches all the songs I listened to on Spotify yesterday and stores them in a local database.

In [ ]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker
import pandas as pd
import tensorflow as tf
import numpy as ny
import requests
import json
import sqlite3
from datetime import datetime
import datetime
from pandas.io.json import json_normalize

DATABASE_LOCATION = "sqlite:///my_played_tracks.sqlite"
USER_ID = "s3mvgio1lbwbi82fndkykla9c"
TOKEN = "BQCKJ7Zj3CVqzSRx9RD4rMQTMtrkvFoqUHkOlSgjNyoTizHyywseurUvYuwGNept-nRgIpPAbnFjnSRB3nrevSf4OO7mtPMSgE78hJj_F-xwwH3ofhpSGT2tqsC7PI7ihj_IDrtuHgzMKamI30_Pr-gUKOkn4x5-qfUKwzSmKl-_Yw"
# Generate a token at: https://api.spotify.com/v1/me/player/recently-played

# Validate the data


def check_if_valid_data(df: pd.DataFrame) -> bool:
    # Check if dataframe is empty
    if df.empty:
        print("This is strange.. No songs were downloaded. Finishing execution..")
        return False

    # Primary Key Check - WE DON'T WANT DUPLICATES
    if pd.Series(df["played_at"]).is_unique:
        pass
        # because you can't listen at two songs at the same time
    else:
        raise Exception("Primary Key Check Failed!!")

    # Check for null values
    if df.isnull().values.any():
        raise Exception("Null values found")

    # Check that all timestamps are from yesterday's date
    yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
    yesterday = yesterday.replace(hour=0, minute=0, second=0, microsecond=0)

    timestamps = df["timestamp"].tolist()
    for timestamp in timestamps:
        if datetime.datetime.strptime(timestamp, "%Y-%m-%d") != yesterday:
            raise Exception(
                "At least one of the returned songs does not come from within the last 24hrs")
    return True


if __name__ == "__main__":

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization":  "Bearer {token}".format(token=TOKEN)
    }

    today = datetime.datetime.now()
    yesterday = today - datetime.timedelta(days=1)
    yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000

    r = requests.get("https://api.spotify.com/v1/me/player/recently-played?after{time}".format(
        time=yesterday_unix_timestamp), headers=headers)

    data = r.json()

    song_names = []
    # featured_artists_names = []
    album_names = []
    artist_names = []
    song_popularity = []
    song_explicit = []
    song_duration_ms = []
    played_at_list = []
    timestamps = []

    # getting songs from the items. The going down the data heirarchy to get certain fields (e.g items > track > name or track > album > artists)

    for song in data["items"]:
        song_names.append(song["track"]["name"])
        artist_names.append(song["track"]["album"]["artists"][0]["name"])
        # featured_artists_names.append(song["track"]["artists"][-1]["name"])
        song_popularity.append(song["track"]["popularity"])
        song_explicit.append(song["track"]["explicit"])
        # TODO: convert the duration from miniseconds to Min:Sec time
        song_duration_ms.append(song["track"]["duration_ms"])

        played_at_list.append(song["played_at"])
        timestamps.append(song["played_at"][0: 10])

    song_dict = {
        "song_name": song_names,
        "artist_name": artist_names,
        "song_popularity": song_popularity,
        "song_explicit": song_explicit,
        "song_duration_ms": song_duration_ms,
        "played_at": played_at_list,
        "timestamp": timestamps
    }

    song_df = pd.DataFrame(song_dict, columns=[
        "song_name", "artist_name", "song_popularity", "song_explicit", "song_duration_ms", "played_at", "timestamp"])

    print(song_df)
    # Validate
    if check_if_valid_data(song_df):

        print("Data is valid! Proceed to the Load stage")


                                  song_name  ...   timestamp
0                                   Casting  ...  2021-09-15
1   Fólk fær andlit (Arr. Jarkko Riihimäki)  ...  2021-09-15
2              The Beach - from I Never Cry  ...  2021-09-15
3                            Bathroom Dance  ...  2021-09-15
4                     Hymn - Mvmt 2: Lament  ...  2021-09-15
5                        Marche - Agincourt  ...  2021-09-15
6                                  Antiphon  ...  2021-09-15
7          Ballade in B-flat Minor: Descent  ...  2021-09-15
8           Song of Hal: Strings in B Minor  ...  2021-09-15
9                                  Antiphon  ...  2021-09-15
10                                 Antiphon  ...  2021-09-15
11                                 Antiphon  ...  2021-09-15
12                                 Antiphon  ...  2021-09-15
13                                 Antiphon  ...  2021-09-15
14                                 Antiphon  ...  2021-09-15
15                      

Exception: ignored